In [1]:
using Pkg
Pkg.activate(".")

  Activating new environment at `~/Documents/Github2/Physics-215-Julia/Session 4 - Fast function calls/Project.toml`


### KR1: Benchmarked at least two (2) different implementation of the same function or process (e.g. raising each element of an array to some power `p`, random array may be used) that utilizes some parameter that can be considered a constant or declared globally. Typical methods: (1) Global variable, (2) Constant global variable, and (3) Named parameter variable.

In [2]:
p = 2 

function multiply_array(x::Vector{Float64})
    s =  zero(eltype(x))
    for y in x
        s = s + y*p
    end
    return s
end

multiply_array (generic function with 1 method)

In [3]:
const p2 = 2

function multiply_array2(x::Vector{Float64})
    s = zero(eltype(x))
    for y in x
        s = s + y*p2
    end
    return s
end

multiply_array2 (generic function with 1 method)

In [4]:
random_array = rand(1000);

In [5]:
using BenchmarkTools

In [6]:
@benchmark multiply_array(random_array)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   58.078 μs …  45.712 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     118.436 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   256.450 μs ± 874.973 μs  ┊ GC (mean ± σ):  1.40% ± 2.88%

  ██▅▄▃▂▂▁▁▁                                                    ▂
  █████████████▇▇▆▇▆▇▅▆▅▆▆▆▆▆▆▆▅▆▆▄▄▄▅▄▅▆▆▄▅▅▄▄▅▄▄▃▅▅▅▄▄▃▄▅▃▃▁▄ █
  58.1 μs       Histogram: log(frequency) by time       3.35 ms <

 Memory estimate: 46.88 KiB, allocs estimate: 3000.

In [7]:
@benchmark multiply_array2(random_array)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.106 μs … 439.429 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.156 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.704 μs ±   6.265 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▁                                                          ▁
  ██▇▅▇██▇▇▇▇▆▇▇▇▇▆▆▆▇▇▆▆▆▆▅▆▅▆▅▆▆▆▆▅▅▅▄▄▆▄▅▄▄▅▄▄▄▄▄▄▄▅▄▃▅▄▁▄ █
  1.11 μs      Histogram: log(frequency) by time      11.6 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

### KR2 Replicated the naive implementation of the polynomial in the textbook.

In [8]:
function naive_polynomial(x, a...) #can i change a... into vector{a}
    p = zero(x)
    for i = 1:length(a)
        p = p + a[i] * x^(i-1)
    end
    return p
end

naive_polynomial (generic function with 1 method)

In [9]:
f_naive(x) = naive_polynomial(x, 1,2,3,4,5,6,7,8,9)

f_naive (generic function with 1 method)

In [10]:
x = 3.5
f_naive(x)

271125.95703125

In [18]:
time_naive = @benchmark f_naive($x)

BenchmarkTools.Trial: 10000 samples with 737 evaluations.
 Range (min … max):  171.342 ns … 335.125 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     258.493 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   511.756 ns ±   3.967 μs  ┊ GC (mean ± σ):  0.33% ± 1.74%

  ▆█▅▄▃▃▂▂▁▁▁▁▁▁                                                ▂
  █████████████████▇▇▇▇▇▆▇▆▆▆▆▆▇▆▆▄▆▅▆▆▅▅▅▅▃▆▅▅▄▅▄▄▅▅▁▃▄▄▄▁▅▄▄▄ █
  171 ns        Histogram: log(frequency) by time       3.67 μs <

 Memory estimate: 32 bytes, allocs estimate: 2.

### KR3: Replicated the naive implementation of the Horner’s method for the same polynomial.

In [12]:
function poly_horner(x, a...)
    b = zero(x)
    for i = length(a):-1:1
        b = a[i] + b * x
    end
    return b
end

poly_horner (generic function with 1 method)

In [13]:
f_horner_naive(x) = poly_horner(x,1,2,3,4,5,6,7,8,9)

f_horner_naive (generic function with 1 method)

In [19]:
time_horner = @benchmark f_horner_naive($x)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):   4.887 ns …   8.941 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):      8.522 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   14.938 ns ± 160.079 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆▆▃█▆              ▁                                         ▂
  ██████▇▆▅▄▄▆▆▆█▅▅▅▇█▆▄▅▅▆▆▅▄▅▄▄▄▅▄▄▁▄▃▅▄▄▁▃▄▃▄▄▁▄▃▄▄▁▄▃▄▄▄▁▄ █
  4.89 ns       Histogram: log(frequency) by time        64 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [15]:
macro horner(x,p...)
    ex = esc(p[end])
    for i = length(p):-1:1
        ex = :(muladd(t, $ex, $(esc(p[i]))))
    end
    Expr(:block, :(t = $(esc(x))), ex)
end


@horner (macro with 1 method)

In [16]:
f_horner_macro(x) = @horner(x, 1,2,3,4,5,6,7,8,9)

f_horner_macro (generic function with 1 method)

In [25]:
time_horner_macro = @benchmark f_horner_macro($x)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  0.045 ns … 212.760 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     0.065 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   0.107 ns ±   2.367 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

             ▂           █                                     
  ▂▃▂▂▄▅▅▅▆▅▆█▅▆▆▆▆▆▆▆▇▇██▇▆▅▅▄▃▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  0.045 ns        Histogram: frequency by time       0.109 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

### KR5: Table showing how many minutes will the function evaluations in both KR3 and KR4 be reduced if KR2 requires 24 hours of runtime.

In [27]:
med_time_naive = median(time_naive.times)
med_time_horner = median(time_horner.times)
med_time_horner_macro = median(time_horner_macro.times);

In [32]:
using DataFrames

In [40]:
speedup1 = med_time_naive / med_time_horner
speedup2 = med_time_naive / med_time_horner_macro

table = DataFrame("Method"=>["Naive", "Horner", "Horner Macro"], "Raw Time (ns)"=>[med_time_naive,med_time_horner,med_time_horner_macro],
"Speed comparison"=>[1.0, speedup1, speedup2]);

In [41]:
print(table)

3×3 DataFrame
 Row │ Method        Raw Time (ns)  Speed comparison 
     │ String        Float64        Float64          
─────┼───────────────────────────────────────────────
   1 │ Naive               258.493            1.0
   2 │ Horner                8.522           30.3324
   3 │ Horner Macro          0.065         3976.81

We see from the summarized table above that the Horner's method performs 30x faster compared to the naive polynomial method, while the macro version of the Horner's method is almost 4000x faster that the naive one. Now assuming that our naive implementation of the polynomial takes 24 hours (1440 minutes) to run, let us check how many minutes it would take for the horner's method and its macro version to finish the same task.

In [43]:
table2 = DataFrame("Method"=>["Naive", "Horner", "Horner Macro"], 
                    "Time (hours)"=>[24, 24/speedup1, 24/speedup2], 
                    "Time (mins)"=>[24*60, 24*60/speedup1, 24*60/speedup2]);

In [44]:
print(table2)

3×3 DataFrame
 Row │ Method        Time (hours)  Time (mins) 
     │ String        Float64       Float64     
─────┼─────────────────────────────────────────
   1 │ Naive          24.0         1440.0
   2 │ Horner          0.791234      47.474
   3 │ Horner Macro    0.00603499     0.362099

We see that using the naive implementation of the Horner's method would finish running after ~47.5 minutes or less than an hour while the macro implementation of the same method would be done in less than a minute (~22 seconds).